In [180]:
from gensim import models, corpora, similarities
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from nltk.tokenize import TweetTokenizer
import csv
import multiprocessing

cores = multiprocessing.cpu_count()
print cores

8


In [181]:
with open('questions.csv', 'rU') as f:
    f.next()
    reader = csv.reader(f)
    qpairs = list(reader)
   
tknzr = TweetTokenizer()
qcorp = []
qdict = {}
for qpair in qpairs:
    qid = qpair[1]
    qwords = tknzr.tokenize(qpair[3])
    q = TaggedDocument(words=qwords, tags=["QID_"+str(qid)])
    qcorp.append(q)
    qdict["QID_"+str(qid)] = qwords
    qid = qpair[2]
    qwords = tknzr.tokenize(qpair[4])
    q = TaggedDocument(words=qwords, tags=["QID_"+str(qid)])
    qcorp.append(q)
    qdict["QID_"+str(qid)] = qwords

In [182]:
model = Doc2Vec(size=100, window=5, negative=5, sample=1e-4, min_count=2, workers=cores, alpha=0.025, min_alpha=0.025)
model.build_vocab(qcorp)
for epoch in range(10):
    print "iteration "+str(epoch+1)
    model.train(qcorp)
    model.alpha -= 0.002
    model.min_alpha = model.alpha
    model.train(qcorp)

iteration 1
iteration 2
iteration 3
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10


In [183]:
model.save("doc2vec.model")

In [184]:
vectors = model.docvecs

In [185]:
for i in range(len(qpairs)):
    qid1 = "QID_"+str(qpairs[i][1])
    qid2 = "QID_"+str(qpairs[i][2])
    score = vectors.similarity(qid1, qid2)
    qpairs[i].extend([score])

In [190]:
for i in range(25,50):
    print qpairs[i]

['25', '51', '52', 'What are some tips on making it through the job interview process at Medicines?', 'What are some tips on making it through the job interview process at Foundation Medicine?', '0', 0.86943831944632222]
['26', '53', '54', 'What is web application?', 'What is the web application framework?', '0', 0.95722550998466083]
['27', '55', '56', 'Does society place too much importance on sports?', 'How do sports contribute to the society?', '0', 0.32875073729150761]
['28', '57', '58', 'What is best way to make money online?', 'What is best way to ask for money online?', '0', 0.74854103355402346]
['29', '59', '60', 'How should I prepare for CA final law?', 'How one should know that he/she completely prepare for CA final exam?', '1', 0.20658843522798939]
['30', '61', '62', "What's one thing you would like to do better?", "What's one thing you do despite knowing better?", '0', 0.55553240180896801]
['31', '63', '64', 'What are some special cares for someone with a nose that gets stu

In [187]:
wordvectors = model.wv

In [192]:
print wordvectors.most_similar(["tell"])

[(u'know', 0.9086978435516357), (u'say', 0.9039154052734375), (u'see', 0.899399995803833), (u'forget', 0.8749495148658752), (u'ask', 0.8747535943984985), (u'talk', 0.8610190153121948), (u'understand', 0.8600683212280273), (u'make', 0.8573578596115112), (u'find', 0.8545108437538147), (u'read', 0.8426806926727295)]


In [189]:
test = "QID_15"
tid = int(test[4:]) - 1
print qcorp[tid]
for q in vectors.most_similar(["QID_15"]):
    print q, qdict[q[0]]

TaggedDocument([u'How', u'can', u'I', u'be', u'a', u'good', u'geologist', u'?'], ['QID_15'])
('QID_149134', 0.7738628387451172) [u'How', u'can', u'I', u'be', u'good', u'programer', u'?']
('QID_627259', 0.7621285319328308) [u'How', u'can', u'I', u'be', u'a', u'good', u'software', u'engineer', u'?']
('QID_16', 0.7554548978805542) [u'What', u'should', u'I', u'do', u'to', u'be', u'a', u'great', u'geologist', u'?']
('QID_604540', 0.7509888410568237) [u'How', u'can', u'I', u'be', u'good', u'engineer', u'?']
('QID_71714', 0.7502573728561401) [u'How', u'can', u'I', u'be', u'a', u'good', u'chess', u'player', u'?']
('QID_410466', 0.7500404119491577) [u'How', u'can', u'I', u'be', u'a', u'good', u'husband', u'for', u'my', u'wife', u'?']
('QID_557277', 0.7480751872062683) [u'How', u'can', u'I', u'be', u'good', u'programer', u'?']
('QID_588867', 0.7438502907752991) [u'How', u'can', u'I', u'be', u'a', u'good', u'husband', u'for', u'my', u'wife', u'?']
('QID_384999', 0.7418942451477051) [u'How', u'can